
## Language Models 

### 1.1 Pre-Processing

#### A. Pad each Sentence

In [1]:
%cd  /Users/MarianaMaroto/Desktop/MS in Data Science/74040 NLP/Homework 1/

# open the input file for reading and create a new output file for writing
readfile = open('train.txt', 'r',  errors="ignore")
writefile = open('train1.txt', 'w')

# read each line in the input file
for line in readfile:
    # remove the trailing carriage return
    line = line.strip()
    writefile.write('<s>' + ' ' + line + ' ' + '</s>' + '\n')
writefile.close()
readfile.close()
    
# open the input file for reading and create a new output file for writing
readfile2 = open('test.txt', 'r',  errors="ignore")
writefile2 = open('test1.txt', 'w')

# read each line in the input file
for line in readfile2:
    # remove the trailing carriage return
    line = line.strip()
    writefile2.write('<s>' + ' ' + line + ' ' + '</s>' + '\n')
writefile2.close()
readfile2.close()

C:\Users\MarianaMaroto\Desktop\MS in Data Science\74040 NLP\Homework 1


#### B. Lowercase all words

In [2]:
# open the input file for reading and create a new output file for writing
readfile = open('train1.txt', 'r',  errors="ignore")
writefile = open('train2.txt', 'w')

# read each line in the input file
for line in readfile:
    line = line.lower()
    writefile.write(line)
writefile.close()   
readfile.close()

# open the input file for reading and create a new output file for writing
readfile2 = open('test1.txt', 'r',  errors="ignore")
writefile2 = open('test2.txt', 'w')

# read each line in the input file
for line in readfile2:
    line = line.lower()
    writefile2.write(line)
writefile2.close()
readfile2.close()

#### C.  Replace all words occurring in the training data once with the token unk. Every word in the test data not seen in training should be treated as unk.

In [3]:
# Open the file in read mode
text = open("train2.txt", "r", errors="ignore")
  
# Create an empty dictionary
d = dict()
  
# Loop through each line of the file
for line in text:
    # Split the line into words
    words = line.split(" ")
    for word in words:
        # Check if the word is already in dictionary
        if word in d:
            # Increment count of word by 1
            d[word] = d[word] + 1
        else:
            # Add the word to dictionary with count 1
            d[word] = 1
            
d_once = dict((k, v) for k, v in d.items() if v == 1)

In [4]:
# open the input file for reading and create a new output file for writing
readfile = open('train2.txt', 'r',  errors="ignore")
writefile = open('train3.txt', 'w')

# read each line in the input file
for line in readfile:
# Split the line into words
    words = line.split(" ")
    for word in words:
        if word in d_once:
            line = line.replace(' ' + word + ' ',' ' + '<unk>'+ ' ')
    writefile.write(line)
writefile.close()   
readfile.close()
    
# open the input file for reading and create a new output file for writing
readfile2 = open('test2.txt', 'r',  errors="ignore")
writefile2 = open('test3.txt', 'w')

# read each line in the input file
for line in readfile2:
# Split the line into words
    words = line.split(" ")
    for word in words:
        if word not in d or word in d_once:
            line = line.replace(' ' + word + ' ',' ' + '<unk>'+ ' ')
    writefile2.write(line)
writefile2.close()   
readfile2.close()

### 1.2 Training the Models

#### A. A unigram maximum likelihood model.

In [5]:
from math import log2

def CountUnigrams(file):
    """
    Count number of unigrams in a file
    """
    text = open(file, "r", errors="ignore")
    counts = dict()

    # Loop through each line of the file
    for line in text:
        # Split the line into words
        words = line.split(" ")
        for unigram in words[1:]:
            # skipping first one which is <s>
            # Check if the word is already in dictionary
            if unigram in counts:
                # Increment count of word by 1
                counts[unigram] = counts[unigram] + 1
            else:
                # Add the word to dictionary with count 1
                counts[unigram] = 1
    return counts
            

def UnigramMLE(train_counts, test_counts):
    """
    Estimate the probabilities of each word
    """
    probs = dict()
    vocab_size = len(train_counts)
    token_count = sum(train_counts.values())
    
    for unigram, unigram_count in train_counts.items():
        prob_nom = unigram_count
        prob_denom = token_count
        probs[unigram] = prob_nom / prob_denom
        
    test_log_likelihood = 0
    for n_gram, n_gram_count in test_counts.items():
        if n_gram not in probs.keys():
            prob = 0.000000000000001
        else:
            prob = probs[n_gram]
        log_likelihood = n_gram_count * log2(prob)
        test_log_likelihood += log_likelihood
    avg_test_log_likelihood = test_log_likelihood / sum(test_counts.values())
    perplexity = 2**(-avg_test_log_likelihood)
    return avg_test_log_likelihood, perplexity

#### B. A bigram maximum likelihood model.

In [6]:
def CountBigrams(file):
    """
    Count number of bigrams in a file
    """
    text = open(file, "r", errors="ignore")
    unigram_counts = dict()
    bigram_counts = dict()

    # Loop through each line of the file
    for line in text:
        # Split the line into words
        words = line.split(" ")
        for unigram in words[1:]:
            # skipping first one which is <s>
            # Check if the word is already in dictionary
            if unigram in unigram_counts:
                # Increment count of word by 1
                unigram_counts[unigram] = unigram_counts[unigram] + 1
            else:
                # Add the word to dictionary with count 1
                unigram_counts[unigram] = 1
                
        for i in range(len(words)-1)[1:]:
        # skipping first one <s>, therefore adding 1: at the end
        # Check if the word is already in dictionary
            bigram = (words[i], words[i+1])
            if bigram in bigram_counts:
                # Increment count of word by 1
                bigram_counts[bigram] = bigram_counts[bigram] + 1
            else:
                # Add the word to dictionary with count 1
                bigram_counts[bigram] = 1
    
    return unigram_counts, bigram_counts
            

def BigramsMLE(uni_train_counts, bi_train_counts, bi_test_counts):
    """
    Estimate the probabilities of each bigram
    """
    probs = dict()
    
    for bigram, bigram_count in bi_train_counts.items():
        word1 = bigram[0]
        prob_nom = bigram_count
        prob_denom = uni_train_counts.get(word1)
        probs[bigram] = prob_nom / prob_denom
        
    test_log_likelihood = 0
    for n_gram, n_gram_count in bi_test_counts.items():
        if n_gram not in probs.keys():
            prob = 0.000000000000001
        else:
            prob = probs[n_gram]
        log_likelihood = n_gram_count * log2(prob)
        test_log_likelihood += log_likelihood
    avg_test_log_likelihood = test_log_likelihood / sum(bi_test_counts.values())
    perplexity = 2**(-avg_test_log_likelihood)
    return avg_test_log_likelihood, perplexity

#### C. A bigram model with Add-One smoothing.

In [7]:
def BigramsMLEAddOneSmooth(uni_train_counts, bi_train_counts, bi_test_counts):
    """
    Estimate the probabilities of each bigram
    """
    probs = dict()
    vocab_size = len(bi_train_counts)
    token_count = sum(bi_train_counts.values())
    
    for bigram, bigram_count in bi_train_counts.items():
        word1 = bigram[0]
        prob_nom = bigram_count + 1
        prob_denom = uni_train_counts.get(word1) + vocab_size
        probs[bigram] = prob_nom / prob_denom
    
    test_log_likelihood = 0
    for n_gram, n_gram_count in bi_test_counts.items():
        if n_gram not in probs.keys():
            test_word1 = n_gram[0]
            prob = 1 / uni_train_counts.get(test_word1) + vocab_size
        else:
            prob = probs[n_gram]
        log_likelihood = n_gram_count * log2(prob)
        test_log_likelihood += log_likelihood
    avg_test_log_likelihood = test_log_likelihood / sum(bi_test_counts.values())
    perplexity = 2**(-avg_test_log_likelihood)
    return avg_test_log_likelihood, perplexity

#### D. A bigram model with discounting and Katz backoff. Discount constant of 0.5.

In [8]:
def BigramsMLEKatzBackoff(uni_train_counts, bi_train_counts, bi_test_counts, c = 0.5):
    """
    Estimate the probabilities of each bigram
    """
    probs = dict()
    vocab_size = len(uni_train_counts)
    token_count = sum(uni_train_counts.values())
    
    for bigram, bigram_count in bi_train_counts.items():
        word1 = bigram[0]
        prob_nom = bigram_count
        prob_denom = uni_train_counts.get(word1)
        probs[bigram] = prob_nom / prob_denom

    test_log_likelihood = 0
    for n_gram, n_gram_count in bi_test_counts.items():
        if n_gram not in probs.keys():
            test_word1 = n_gram[0]
            prob = c * (uni_train_counts.get(test_word1) / token_count)
        else:
            prob = probs[n_gram]
        log_likelihood = n_gram_count * log2(prob)
        test_log_likelihood += log_likelihood
    avg_test_log_likelihood = test_log_likelihood / sum(bi_test_counts.values())
    perplexity = 2**(-avg_test_log_likelihood)
    return avg_test_log_likelihood, perplexity

### Compute on test data

In [18]:
# Unigram Model 
uni_train_counts = CountUnigrams('train3.txt')
uni_test_counts = CountUnigrams('test3.txt')
print(UnigramMLE(uni_train_counts, uni_test_counts))

#Bigram Model
uni_train_counts, bi_train_counts = CountBigrams('train3.txt')
uni_test_counts, bi_test_counts = CountBigrams('test3.txt')
print(BigramsMLE(uni_train_counts, bi_train_counts, bi_test_counts))

#Bigram Model with Add One Smoothening
uni_train_counts, bi_train_counts = CountBigrams('train3.txt')
uni_test_counts, bi_test_counts = CountBigrams('test3.txt')
print(BigramsMLEAddOneSmooth(uni_train_counts, bi_train_counts, bi_test_counts))

#Bigram Model with KatzBackoff
uni_train_counts, bi_train_counts = CountBigrams('train3.txt')
uni_test_counts, bi_test_counts = CountBigrams('test3.txt')
print(BigramsMLEKatzBackoff(uni_train_counts, bi_train_counts, bi_test_counts))

(-10.099596076139623, 1097.188793154414)
(-16.005691885679507, 65795.07085373304)
(-6.292954575609962, 78.40939242153524)
(-8.118504031545793, 277.9157971414787)
